In [1]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 563 kB 9.3 MB/s 
  Attempting uninstall: transformers
    Found existing installation: transformers 2.7.0
    Uninstalling transformers-2.7.0:
      Successfully uninstalled transformers-2.7.0
     |████████████████████████████████| 159 kB 9.2 MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=040ae8616aeae4cd7b56f0b478c1243527cd7b713b051e4d1fe8e139106f6194
  Stored in directory: /root/.cache/pip/wheels/1f/1b/a6/a808a7e4d1f7584e42f5e279664cd48bf24ed8392218ce6be4
Successfully built seqeval


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


train_df = pd.read_csv('../input/midas-task/reddit_data.csv')
train_df.dropna(inplace=True)
le = preprocessing.LabelEncoder()
le.fit(train_df["flair"])
train_df["label"] = le.transform(train_df["flair"])
print(le.classes_)

train_df = train_df[['dirty_text', 'label']]

train, validate = np.split(train_df.sample(frac=1), [int(.8*len(train_df))])


['AskIndia' 'Business/Finance' 'Coronavirus' 'Non-Political'
 'Policy/Economy' 'Politics' 'Science/Technology']


In [3]:
len(validate), len(train), len(train_df)

(15465, 61856, 77321)

In [4]:
from simpletransformers.classification import ClassificationModel

custom_args = {"train_batch_size":32, 
                    "fp16":False, 
                    "do_lower_case": True, 
                    "save_eval_checkpoints": False,
                    'save_model_every_epoch': False, 
                    'gradient_accumulation_steps': 2, 
                    'learning_rate':1e-3, 
                    'num_train_epochs': 5, 
                    'reprocess_input_data': True, 
                    'overwrite_output_dir': True}

model = ClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=7, args=custom_args)


In [5]:
model.train_model(train)

/opt/conda/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 1.367548

/opt/conda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.566715


Running loss: 1.687253


Running loss: 1.820052

In [6]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(validate, f1=f1_multiclass, acc=accuracy_score)

/opt/conda/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [7]:
print(result)

{'mcc': 0.0, 'f1': 0.29673456191399933, 'acc': 0.29673456191399933, 'eval_loss': 1.7264134946135725}


In [8]:
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=7, args=custom_args)
model.train_model(train)

Running loss: 1.526704


Running loss: 1.662163

In [9]:
result, model_outputs, wrong_predictions = model.eval_model(validate, f1=f1_multiclass, acc=accuracy_score)

In [10]:
print(result)

{'mcc': 0.0, 'f1': 0.26608470740381507, 'acc': 0.26608470740381507, 'eval_loss': 1.7352961404123868}


In [11]:
import transformers
transformers.__version__

'2.8.0'